In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np

# Loading CSV file

In [2]:
df_anime = pd.read_csv(r'data\myAnimelist-No_Hentai 2.csv')
df_anime.rename(columns={"animeID":'anime_id'},inplace=True)
df_rating = pd.read_csv(r'data\rating.csv')

In [20]:
print(df_rating.shape)
# filtering for computational reasons & file saving size
df_rating_filtered = df_rating[df_rating.groupby("user_id")['user_id'].transform('size') > 500]
df_rating_filtered

(7813737, 3)


,user_id,anime_id,rating
1614,17,6,7
1615,17,19,10
1616,17,30,9
1617,17,32,10
1618,17,43,8
...,...,...,...
7812698,73502,18893,9
7812699,73502,19151,5
7812700,73502,19221,6
7812701,73502,19367,6


In [3]:
display(df_anime)
display(df_rating)

,anime_id,name,type,source,episodes,duration,genre,rating,score,scored_by,rank,popularity,members,favorites
0,1,Cowboy Bebop,TV,Original,26,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460
1,5,Cowboy Bebop: Tengoku no Tobira,Movie,Original,1,1:55:00,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",R - 17+ (violence & profanity),8.41,120243,164,449,197791,776
2,6,Trigun,TV,Manga,26,0:24:00,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...",PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432
3,7,Witch Hunter Robin,TV,Original,26,0:25:00,"['Action', 'Magic', 'Police', 'Supernatural', ...",PG-13 - Teens 13 or older,7.33,32837,2371,1171,79397,537
4,8,Bouken Ou Beet,TV,Manga,52,0:23:00,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",PG - Children,7.03,4894,3544,3704,11708,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5555,38923,Sinbi Apateu: Ghost Ball Bimil,TV,Original,24,0:24:00,"['Action', 'Mystery', 'Kids']",G - All Ages,7.00,4,13056,15281,30,1
5556,38961,Takoyaki Story,Music,Original,1,0:01:00,"['Music', 'Dementia']",R+ - Mild Nudity,3.28,25,13254,15083,46,0
5557,38985,Birthday Wonderland,Movie,Book,1,1:55:00,"['Adventure', 'Fantasy', 'Kids']",G - All Ages,6.61,327,5504,9195,835,2
5558,39022,Ling Feng Zhe,ONA,Original,7,0:24:00,"['Slice of Life', 'Historical', 'Drama', 'Roma...",PG-13 - Teens 13 or older,6.08,38,11923,10455,467,2


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [4]:
# Database Path object for easier calling
db_path = r"db\anime.db"

# Create a new database

In [5]:
# Create a new database file

def create_database(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    # connect to database file
    try:
        conn = sqlite3.connect(db_file)
        print(f'sqlite3 version: {sqlite3.version}')
        print('Created Database')
    # print error if connection can not be established
    except Error as error:
        print(error)
        print('Could not create Database')
    # close connection
    finally:
        if conn:
            conn.close()
    

# run the above function (save database file to path)
if __name__ == '__main__':
    create_database(db_path)

sqlite3 version: 2.6.0
Created Database


In [6]:
# # Create a new database that resides in the memory (RAM)

# def create_database():
#     """ create a database connection to a database that resides
#         in the memory
#     """
#     conn = None;
#     # connect to memory
#     try:
#         conn = sqlite3.connect(':memory:')
#         print(f'sqlite3 version: {sqlite3.version}')
#         print('Created Database')
#     # print error if connection can not be established
#     except Error as error:
#         print(error)
#         print('Could not create Database')
#     # close connection
#     finally:
#         if conn:
#             conn.close()
    

# # run the above function
# if __name__ == '__main__':
#     create_database()


# Create Tables

In [7]:
# function for establish connection to SQLite3 for editing database

def establish_connection(db_file):
    """ 
    creates a database connection to a SQLite database file
    
    :returns: Connection object or None
    """
    conn = None
    # connect to database file
    try:
        conn = sqlite3.connect(db_file)
        print(f'sqlite3 version: {sqlite3.version}')
        print('Connected to Database path}')
    # print error if connection can not be established
    except Error as error:
        print(error)
        print('Could not create Database')
    
    return conn

In [8]:
# function for creating table in database

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    print('Creating tables')
    try:
        # make object cursor from connect
        c = conn.cursor()
        # from cursor use execute to create table
        c.execute(create_table_sql)
    except Error as error:
        print(error)

In [10]:
# Create database schema

def main():
    
    sql_create_anime_table = """ CREATE TABLE IF NOT EXISTS Anime (
                                        anime_id integer PRIMARY KEY NOT NULL,
                                        name text NOT NULL,
                                        type text,
                                        source text,
                                        episodes integer,
                                        duration datetime,
                                        genre text,
                                        rating text,
                                        score real,
                                        scored_by integer,
                                        rank integer,
                                        popularity integer,
                                        members integer,
                                        favorites integer,
                                        FOREIGN KEY (anime_id) REFERENCES rating (anime_id)
                                    ); """

    sql_create_rating_table = """CREATE TABLE IF NOT EXISTS Rating (
                                    anime_id integer PRIMARY KEY NOT NULL,
                                    user_id integer NOT NULL,
                                    rating integer,
                                    FOREIGN KEY (anime_id) REFERENCES anime (anime_id)
                                    
                                );"""

    # create a database connection
    conn = establish_connection(db_path)

    # create tables
    if conn is not None:
        # create Anime table
        create_table(conn, sql_create_anime_table)

        # create Rating table
        create_table(conn, sql_create_rating_table)
        
        print("Created tables")
    else:
        print("Error! cannot create the database connection.")
        
    

    
if __name__ == '__main__':
    main()

sqlite3 version: 2.6.0
Connected to Database path}
Creating tables
Creating tables
Created tables


In [11]:
display(df_anime.head(5))
display(df_rating.head(5))

,anime_id,name,type,source,episodes,duration,genre,rating,score,scored_by,rank,popularity,members,favorites
0,1,Cowboy Bebop,TV,Original,26,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460
1,5,Cowboy Bebop: Tengoku no Tobira,Movie,Original,1,1:55:00,"['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']",R - 17+ (violence & profanity),8.41,120243,164,449,197791,776
2,6,Trigun,TV,Manga,26,0:24:00,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...",PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432
3,7,Witch Hunter Robin,TV,Original,26,0:25:00,"['Action', 'Magic', 'Police', 'Supernatural', ...",PG-13 - Teens 13 or older,7.33,32837,2371,1171,79397,537
4,8,Bouken Ou Beet,TV,Manga,52,0:23:00,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",PG - Children,7.03,4894,3544,3704,11708,14


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


# Populating Tables from csv files

In [ ]:
# exporting data from dataframe 'anime' to table anime
df_anime.to_sql('Anime',sqlite3.connect(db_path), if_exists='replace')

# export data from dataframe 'rating' to table rating 
df_rating.to_sql('Rating',sqlite3.connect(db_path), if_exists='replace')

In [ ]:
# close connection to database

def close_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        conn.close()
        print('Connection successfully closed')
    except Error as error:
        print(error)
        print('Could not close connection')
# run the above function (save database file to path)
if __name__ == '__main__':
    close_connection(db_path)

# Querying data with conditions and save to Pandas Dataframe

In [5]:
conn = sqlite3.connect(db_path)

In [6]:

df_score_greater_then_8 = pd.read_sql_query("""SELECT * 
                                               from Anime 
                                               where score>8 
                                               order by score desc;""", conn)
df_score_greater_then_8

,index,anime_id,name,type,source,episodes,duration,genre,rating,score,scored_by,rank,popularity,members,favorites
0,5551,38781,Violence Voyager,Movie,Original,1,1:23:00,"['Adventure', 'Comedy', 'Horror']",R - 17+ (violence & profanity),10.00,1,13544,14765,65,0
1,5318,37149,Xing You Ji: Fengbao Famila,ONA,Original,1,1:10:00,"['Sci-Fi', 'Adventure', 'Fantasy']",PG-13 - Teens 13 or older,9.33,3,13633,12651,171,0
2,5539,38570,Tatakae! Dokan-kun: Robolympic-hen,TV,Original,13,0:05:00,"['Comedy', 'Mecha']",G - All Ages,9.25,4,13288,15264,31,0
3,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
4,2580,9253,Steins;Gate,TV,Visual novel,24,0:24:00,"['Thriller', 'Sci-Fi']",PG-13 - Teens 13 or older,9.14,633590,3,7,1139182,104173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,2616,9513,Beelzebub,TV,Manga,60,0:24:00,"['Action', 'Comedy', 'Demons', 'Supernatural',...",PG-13 - Teens 13 or older,8.01,148483,554,215,332769,4097
437,3971,24921,Aoki Hagane no Arpeggio: Ars Nova Cadenza,Movie,Manga,1,1:45:00,"['Action', 'Sci-Fi', 'Seinen']",PG-13 - Teens 13 or older,8.01,8034,552,2831,20938,105
438,4075,27775,Plastic Memories,TV,Original,13,0:24:00,"['Sci-Fi', 'Drama', 'Romance']",PG-13 - Teens 13 or older,8.01,197098,557,155,396716,6153
439,4297,30415,High☆Speed!: Free! Starting Days,Movie,Light novel,1,1:50:00,"['Slice of Life', 'Comedy', 'Sports', 'Drama',...",PG-13 - Teens 13 or older,8.01,17893,555,1823,44425,202


In [7]:
df_rank_less_then_100 = pd.read_sql_query("Select * from Anime where rank<100 order by rank asc;", conn)


df_rank_less_then_100


,index,anime_id,name,type,source,episodes,duration,genre,rating,score,scored_by,rank,popularity,members,favorites
0,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
1,4555,32281,Kimi no Na wa.,Movie,Original,1,1:46:00,"['Romance', 'Supernatural', 'School', 'Drama']",PG-13 - Teens 13 or older,9.14,588675,2,20,900593,43260
2,2580,9253,Steins;Gate,TV,Visual novel,24,0:24:00,"['Thriller', 'Sci-Fi']",PG-13 - Teens 13 or older,9.14,633590,3,7,1139182,104173
3,4158,28977,Gintama°,TV,Manga,51,0:24:00,"['Action', 'Comedy', 'Historical', 'Parody', '...",R - 17+ (violence & profanity),9.13,82835,4,362,232437,6375
4,2893,11061,Hunter x Hunter (2011),TV,Manga,148,0:23:00,"['Action', 'Adventure', 'Fantasy', 'Shounen', ...",PG-13 - Teens 13 or older,9.12,468889,5,30,840943,76048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,117,170,Slam Dunk,TV,Manga,101,0:23:00,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",PG-13 - Teens 13 or older,8.54,60029,92,751,128264,3750
80,2715,9989,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,TV,Original,11,0:22:00,"['Slice of Life', 'Supernatural', 'Drama']",PG-13 - Teens 13 or older,8.53,439345,93,49,734942,22969
81,10,21,One Piece,TV,Manga,909,0:24:00,"['Action', 'Adventure', 'Comedy', 'Super Power...",PG-13 - Teens 13 or older,8.53,465454,94,36,803871,76869
82,1913,5028,Major S5,TV,Manga,25,0:24:00,"['Comedy', 'Drama', 'Romance', 'Shounen', 'Spo...",PG-13 - Teens 13 or older,8.52,25258,97,1891,42107,479


In [8]:
fullmetal_alchemist_bh_user_ratings = pd.read_sql_query(""" select * from Rating r
                                                            inner join Anime a on a.anime_id = r.anime_id
                                                            where a.anime_id = 5114
                                                            group by user_id
                                                            order by a.anime_id asc

                                                            ;""",conn)
                  
                  
fullmetal_alchemist_bh_user_ratings                      

,index,user_id,anime_id,rating,index,anime_id,name,type,source,episodes,duration,genre,rating,score,scored_by,rank,popularity,members,favorites
0,183,3,5114,10,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
1,1165,10,5114,10,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
2,1250,11,5114,8,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
3,1295,12,5114,9,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
4,1718,17,5114,10,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24569,7811580,73499,5114,10,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
24570,7812050,73500,5114,9,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
24571,7813047,73504,5114,10,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331
24572,7813249,73507,5114,9,1939,5114,Fullmetal Alchemist: Brotherhood,TV,Manga,64,0:24:00,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...",R - 17+ (violence & profanity),9.24,826899,1,4,1355349,120331


In [11]:
most_frequent_raters = pd.read_sql_query(""" select count(r.user_id) total_occurrence,r.user_id, avg(r.rating) avg_rating from Rating r
                                             inner join Anime a on a.anime_id = r.anime_id
                                             group by r.user_id
                                             order by total_occurrence desc

                                             limit 10

                                            ;""",conn)
                  
                            
                  
most_frequent_raters                  
                      

,total_occurrence,user_id,avg_rating
0,4349,48766,-1.000000
1,2250,42635,6.414667
2,1792,57620,7.918527
3,1594,59643,7.199498
4,1504,58343,0.630984
5,1473,45659,7.177868
6,1417,66021,-0.723359
7,1334,52371,0.637181
8,1330,65840,7.546617
9,1330,7345,6.313534


In [10]:
least_frequent_raters = pd.read_sql_query("""select count(r.user_id) total_occurrence,r.user_id, avg(r.rating) avg_rating from Rating r
                                             inner join Anime a on a.anime_id = r.anime_id
                                             group by r.user_id
                                             order by total_occurrence asc

                                             limit 10

                                            ;""",conn)
                  
                            
                  
least_frequent_raters                  
                      

,total_occurrence,user_id,avg_rating
0,1,9,8.0
1,1,59,9.0
2,1,70,10.0
3,1,83,10.0
4,1,113,9.0
5,1,114,8.0
6,1,131,10.0
7,1,195,7.0
8,1,212,9.0
9,1,238,10.0


In [13]:
pd.read_sql_query("""select count(r.user_id) total_occurrence, r.*,
                            a.name,a.anime_id, a.rank from Anime a
                     inner join Rating r on a.anime_id = r.anime_id
                     group by a.anime_id
                     order by total_occurrence desc
                     
                     
                      
                    ;""",conn)
                  
                            
                  
                  
                      

,total_occurrence,index,user_id,anime_id,rating,name,anime_id,rank
0,39340,173,3,1535,10,Death Note,1535,54
1,30583,83,1,11757,10,Sword Art Online,11757,1440
2,29584,105,1,16498,-1,Shingeki no Kyojin,16498,116
3,27718,851,7,1575,9,Code Geass: Hangyaku no Lelouch,1575,32
4,27506,3,1,226,-1,Elfen Lied,226,1067
...,...,...,...,...,...,...,...,...
4462,1,5099301,48766,3414,-1,Hula Kappa,3414,11262
4463,1,5099273,48766,3372,-1,RGB Adventure,3372,12644
4464,1,5099082,48766,3073,-1,Robby to Kerobby,3073,12651
4465,1,5098963,48766,2930,-1,Tensai? Dr. Hamax,2930,13184
